In [19]:
import numpy as np  
import matplotlib.pyplot as plt  
import pandas as pd  
from sklearn.preprocessing import Binarizer, LabelEncoder, OneHotEncoder
from sklearn.cross_validation import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint

In [2]:
data = pd.read_csv('sales_loss_win_data.csv')

In [3]:
data = data.drop('Opportunity Number', axis =1 )

In [4]:
le = LabelEncoder()

In [5]:
class MultiColumnLabelEncoder:
    def __init__(self,columns = None):
        self.columns = columns # array of column names to encode

    def fit(self,X,y=None):
        return self # not relevant here

    def transform(self,X):
        '''
        Transforms columns of X specified in self.columns using
        LabelEncoder(). If no columns specified, transforms all
        columns in X.
        '''
        output = X.copy()
        if self.columns is not None:
            for col in self.columns:
                output[col] = LabelEncoder().fit_transform(output[col])
        else:
            for colname,col in output.iteritems():
                output[colname] = LabelEncoder().fit_transform(col)
        return output

    def fit_transform(self,X,y=None):
        return self.fit(X,y).transform(X)

In [6]:
final_data = MultiColumnLabelEncoder(columns = ['Supplies Subgroup', 'Supplies Group', 'Region', 'Route To Market', 
                                  'Opportunity Result', 'Competitor Type']).fit_transform(data)

## Gradient Boosting

Gradient boosting is a machine learning technique for regression and classification problems, which produces a prediction model in the form of an ensemble of weak prediction models, typically decision trees. It builds the model in a stage-wise fashion like other boosting methods do, and it generalizes them by allowing optimization of an arbitrary differentiable loss function.

Source: https://en.wikipedia.org/wiki/Gradient_boosting


In [7]:
yVar = final_data['Opportunity Result']
xVar = final_data.loc[:, final_data.columns != 'Opportunity Result']

In [8]:
X_train, X_test, y_train, y_test = train_test_split(xVar, yVar, test_size=0.2)
print (X_train.shape, y_train.shape)
print (X_test.shape, y_test.shape)

(62420, 17) (62420,)
(15605, 17) (15605,)


In [30]:
gb = GradientBoostingClassifier()

In [31]:
model = gb.fit(X_train, y_train)

In [35]:
model

GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='deviance', max_depth=3,
              max_features=None, max_leaf_nodes=None,
              min_impurity_split=1e-07, min_samples_leaf=1,
              min_samples_split=2, min_weight_fraction_leaf=0.0,
              n_estimators=100, presort='auto', random_state=None,
              subsample=1.0, verbose=0, warm_start=False)

In [32]:
model.score(X_test, y_test)

0.87587311759051589

In [14]:
y_pred = gb.predict(X_test)  

In [15]:
pd.crosstab(y_test, y_pred, rownames=['Actual Result'], colnames=['Predicted Result'])

Predicted Result,0,1
Actual Result,,
0,11505,616
1,1321,2163


In [41]:
hyperparameters = {
                   "max_depth": randint(3, 6),
                   "max_features": randint(1, 10),
                   "min_samples_leaf": randint(1, 10)
                  }

In [42]:
clf = RandomizedSearchCV(model, hyperparameters, cv=4)

clf.fit(X_train, y_train)

print("GB Score after CV: %s" % clf.score(X_test, y_test))

GB Score after CV: 0.884524190964
